最近总是接触各种
Jupyter…（JupyterNotebook/JupyterHub），发现居然还有可以完全跑在浏览器上的
[Jupyterlite](https://github.com/jupyterlite/jupyterlite)，那是不是可以直接放一个到这儿呢？

## 大致的概念

原来的 JupyterNotebook 是传统的 CS 模式，各种语言（e.g. python）的
kernel 是跑在服务端的

但现在 python（和一些常用的库）也有了 WebAssembly
的移植版（e.g. [pyodide](https://pyodide.org/en/stable/)）， 这样 kernel
也可以在浏览器上实现了
（e.g. [pyolite](https://jupyterlite.readthedocs.io/en/latest/quickstart/using.html#a-python-kernel-powered-by-pyodide)），
因此也就可以创建一个全静态的 site，UI 和 kernel 都在浏览器中运行，
中间用
[mock-socket](https://jupyterlite.readthedocs.io/en/latest/quickstart/using.html#kernels)
假装通讯， 这样就可以把 notebook 那一套 CS 模式移植过来

## 部署流程

### 安装 jupyterlite CLI

`pip install --pre jupyterlite`（现在还处于
[pre-release](https://github.com/jupyterlite/jupyterlite/releases/tag/v0.1.0b12)
的状态）

### 生成 site

引自 jupyterlite CLI
[文档](https://jupyterlite.readthedocs.io/en/latest/reference/cli.html#cli)，

> The jupyter lite (or jupyter-lite) CLI provides tools for lifecycle of
> combining…
>
> -   the core JupyterLite static assets
> -   extra application features like Lab Extensions and settings
> -   kernel-specific resources like Python wheels
> -   user-authored content like Notebooks
>
> … into a ready-to-deploy (and optionally reproducible) Jupyter sites
> which require an HTTP server, but no application server.

主要命令
[`jupyter lite build`](https://jupyterlite.readthedocs.io/en/latest/reference/cli.html#build)，一些重要参数

-   `--config` 可以指定一个 JSON 配置，默认是当前目录下名为
    `jupyter_lite_config.json` 的文件，
    例子可看[这里](https://github.com/jupyterlite/jupyterlite/blob/main/examples/jupyter_lite_config.json)
-   `--lite-dir` 指定
    [LiteDir](https://jupyterlite.readthedocs.io/en/latest/reference/cli.html#the-lite-dir)，
    这个目录主要包含用户提供的内容等
-   `--output-dir` 输出 site 的目录

### 例子

以下是一个 build script 和配置的例子：它会首先创建 python virutal env
然后安装 jupyterlite 最后 build 一个 site 到目录 `jl` 中， 这里根据
[Create a JupyterLite archive that can be used
offline](https://jupyterlite.readthedocs.io/en/latest/howto/configure/advanced/offline.html)
下载完整的 pyodide 以及 mathjax（不需要用
CDN），另外也示范性地添加预下载的 `pyjokes` wheel（不需要从 pypi
下载）， 这样创建出来的 site 即使完全不联网也是能运作的，只是体积比较大

-   `build.sh`

    ``` bash
    #!/bin/bash

    # Directory containing build.sh
    SCRIPT_DIR=$( cd -- "$( dirname -- "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )
    # Python venv
    VENV_DIR=$SCRIPT_DIR/venv
    # Pyodide download path
    PYODIDE_VER=0.21.1
    PYODIDE_PATH=$SCRIPT_DIR/pyodide.tar.bz2
    # Output directory
    OUTPUT_DIR=$SCRIPT_DIR/jl

    echo "==== Script dir: $SCRIPT_DIR"
    echo "==== Python venv dir: $VENV_DIR"
    echo "==== Pyodide version: $PYODIDE_VER"
    echo "==== Pyodide path: $PYODIDE_PATH"
    echo "==== Output dir: $OUTPUT_DIR"

    cd $SCRIPT_DIR

    if [ ! -d $VENV_DIR ]; then
      echo "==== Python venv not found, create a new one"
      python3 -m venv $VENV_DIR
    else
      echo "==== Python venv found"
    fi

    echo "==== Activating venv"
    source $VENV_DIR/bin/activate

    echo "==== Run pip install jupyterlite[piplite,mathjax]"
    pip install --pre jupyterlite[piplite,mathjax]

    if [ ! -f $PYODIDE_PATH ]; then
      echo "==== Pyodide not found, try to download... (or you can manally download it)"
      wget --verbose -O $PYODIDE_PATH \
        https://github.com/pyodide/pyodide/releases/download/${PYODIDE_VER}/pyodide-build-${PYODIDE_VER}.tar.bz2
    else
      echo "==== Pyodide found"
    fi

    echo "==== Run jupyter lite build..."
    jupyter lite build --lite-dir=$SCRIPT_DIR --output-dir=$OUTPUT_DIR --pyodide=$PYODIDE_PATH --config=$SCRIPT_DIR/jupyter_lite_config.json
    ```

-   `jupyter_lite_config.json`

    ``` json
    {
      "LiteBuildConfig": {
        "piplite_urls": [
          "https://files.pythonhosted.org/packages/6a/0e/4a824a4fc93b9725a985fcea7115927b72f2ca6966008300a982cb869720/pyjokes-0.6.0-py2.py3-none-any.whl"
        ]
      }
    }
    ```

### 部署

部署方式有多种，例如可以在 github 上新建一个名为 `jupyterlite`
的项目，把刚才生成的 site push 上去，并且设置 github page，
这样它就可以在 `<username>.github.io/jupyterlite` 这个路径上访问得到了

### 内容

如上所述，可将内容（`*.ipynb`）放置到 LiteDir 的 `files` 目录中，再次跑
build script 即可更新到 site 中

## 使用

### 不同之处

原 CS 模式下，packages 是在服务器上安装好，可以直接 `import`，而在
jupyterlite 中，似乎除 pyodide 自带的那些包外， 需要首先
`piplite.install` 才能 `import`，例如：

In [ ]:
import pyjokes
pyjokes

这是会失败的

In [ ]:
import piplite
await piplite.install('pyjokes')
import pyjokes
pyjokes.get_joke()

这样才行，看看[效果](/jupyterlite/lab?path=posts%2F2022%2Fjupyterlite%2Findex.ipynb)

注：上面将 whl url 写到 `piplite_urls` 中只是将 wheel 提前下载到 site
中，并告诉 jupyterlite 如果要 piplite install 这个包则不要去 pypi
找而是在 site 里找，但要使用它之前还是需要 `piplite.install` 的